# Description of the problem

### I. The problem
#### Mr. A owns a few different restaurants in Auckland but all on the north shore. This year he wants to expand his business portfolio, however he doesn't want to start from scratch but instead he wants to:
#### 1. Find a similar area in Auckland so that his current restaurant style/ segmentation would fit.
#### 2. Compare potential restaurant data from Zomato so he can approach the owner and see if they want to sell
#### Based on the result, he then will decide area/ restaurant he will purchase

### II. Data
##### 1. Auckland suburbs will be scraped from Wikipedia
##### 2. We then use Foursquare to find venues for area analysis
##### 3. Restaurant rating will be scraped from Zomato API for K-means Clustering to recommend similar restaurants


# Project detail

### I. Grab Auckland suburbs

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
from bs4 import BeautifulSoup

Libraries imported.


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_suburbs_of_Auckland'

In [54]:
data = requests.get(url)
soup = BeautifulSoup(data.text, 'html.parser')
## After careful observation, I find that all relevant info embeded between <ul>
df = pd.DataFrame({'suburbs': []})
for ul in soup.find_all('ul'):
    for li in ul.find_all('li'):
        df = df.append({'suburbs': li.text}, ignore_index=True)
        
        
df.head()

,suburbs
0,1 Auckland central
1,2 North Shore
2,3 South Auckland and eastern suburbs\n\n3.1 Pa...
3,3.1 Papakura
4,4 Pukekohe


#### Clean the table

In [60]:
df1=df.iloc[9:311,]
df1.head()

,suburbs
9,Arch Hill
10,Auckland CBD
11,Avondale
12,Balmoral
13,Blackpool


In [ ]:
df1.head()

In [247]:
# delete done column
del df1['done']

#### Get geocode

In [ ]:
## Rename to get geocode
df1.loc[144,'suburbs']='HalfMoon Bay'

#### Loop through data to get geocode set each request =1 with i

In [ ]:
while i<311:    
    i=0
    for index, row in df1.iterrows():
        if row['done']!=1:
            address = row['suburbs'] + ', Auckland'

            geolocator = Nominatim(user_agent="ny_explorer")
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            print('The geograpical coordinate of ',address,' City are {}, {}.'.format(latitude, longitude))
            df1.at[index,'latitude']=latitude
            df1.at[index,'longitude']=longitude
            df1.at[index,'done']=1
            i=i+1
            if i==1:
                break

In [124]:
#check 
df1.head()

,suburbs,latitude,longitude,done
9,Arch Hill,-36.866092,174.745972,1.0
10,Auckland CBD,-36.848911,174.765226,1.0
11,Avondale,-36.893058,174.692814,1.0
12,Balmoral,-36.889205,174.748694,1.0
13,Blackpool,-36.788905,175.012243,1.0


In [125]:
# check for null
df1.isnull().sum()

suburbs      0
latitude     0
longitude    0
done         0
dtype: int64

In [128]:
## Drop some unkown suburb
df1=df1.drop(294)
# save to CSV just in case for latter use
df1.to_csv('out.csv', index=False)  

#### Create a map of Auckland with suburbs superimposed on top.

In [129]:
address = 'Auckland'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [145]:
# create map  using latitude and longitude values
map_1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for index, row in df1.iterrows():
    lat=row['latitude']
    lng=row['longitude']
    label = row['suburbs'],'Auckland'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
       [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_1)  
    
map_1

##### Define Foursquare credentials 

In [209]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID='MAANLALQ0AREGRVQSVSVQPSZZRLCRXOODYCM3SB3MXXAZYS5'
CLIENT_SECRET='KIJZGG4BXA3P2XYVJQFYPO4LH51KUEBMEH3MADNDSCZOELHH'
VERSION = '20180604'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [210]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [212]:
# type your answer here

manhattan_venues = getNearbyVenues(names=df1['suburbs'],
                                   latitudes=df1['latitude'],
                                   longitudes=df1['longitude']
                                  )



Arch Hill
Auckland CBD
Avondale
Balmoral
Blackpool
Blockhouse Bay
Eden Terrace
Eden Valley
Ellerslie
Epsom
Freemans Bay
Glendowie
Glen Innes
Grafton
Greenlane
Greenwoods Corner
Grey Lynn
Herne Bay
Hillsborough
Kingsland
Kohimarama
Lynfield
Meadowbank
Mission Bay
Morningside
Mount Albert
Mount Eden
Mount Roskill
Mount Wellington
Newmarket
Newton
New Windsor
Onehunga
Oneroa
Onetangi
One Tree Hill
Orakei
Oranga
Ostend
Otahuhu
Owairaka
Palm Beach
Panmure
Parnell
Penrose
Point England
Point Chevalier
Ponsonby
Remuera
Royal Oak
Saint Heliers
Saint Johns
Saint Marys Bay
Sandringham
Stonefields
Tamaki
Te Papapa
Three Kings
Waikowhai
Wai o Taiki Bay
Waterview
Western Springs
Westfield
Westmere
Albany
Bayswater
Bayview
Beach Haven
Belmont
Birkdale
Birkenhead
Browns Bay
Campbells Bay
Castor Bay
Chatswood
Cheltenham
Crown Hill
Devonport
Fairview Heights
Forrest Hill
Glenfield
Greenhithe
Hauraki
Highbury
Hillcrest
Long Bay
Mairangi Bay
Marlborough
Milford
Murrays Bay
Narrow Neck
Northcote
Northcote

In [213]:
print(manhattan_venues.shape)
manhattan_venues.head()

(2355, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arch Hill,-36.866092,174.745972,Philippes Chocolat,-36.865305,174.744822,Bakery
1,Arch Hill,-36.866092,174.745972,Crumb,-36.862473,174.745015,Café
2,Arch Hill,-36.866092,174.745972,Charlie Boys,-36.862712,174.748618,Café
3,Arch Hill,-36.866092,174.745972,New Bond Street,-36.868824,174.746846,Bar
4,Auckland CBD,-36.848911,174.765226,Mrs Higgins Oven Fresh Cookies,-36.850006,174.764820,Bakery


In [214]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Albany,12,12,12,12,12,12
Algies Bay,1,1,1,1,1,1
Anawhata,2,2,2,2,2,2
Ararimu,3,3,3,3,3,3
Arch Hill,4,4,4,4,4,4
Ardmore,1,1,1,1,1,1
Arkles Bay,8,8,8,8,8,8
Army Bay,2,2,2,2,2,2
Auckland CBD,100,100,100,100,100,100


In [215]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 238 uniques categories.


### Analyze each suburbs

In [218]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['suburbs'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,suburbs,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Stadium,Bay,Beach,Beer Bar,Beer Garden,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Casino,Castle,Cheese Shop,Child Care Service,Chinese Restaurant,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Spring,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Night Market,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Resort,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Summer Camp,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,Arch Hill,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Arch Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Group rows by suburbs and by taking the mean of the frequency of occurrence of each category

In [229]:
manhattan_grouped = manhattan_onehot.groupby('suburbs').mean().reset_index()
manhattan_grouped.head()

,suburbs,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Stadium,Bay,Beach,Beer Bar,Beer Garden,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Casino,Castle,Cheese Shop,Child Care Service,Chinese Restaurant,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Spring,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Night Market,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Resort,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Summer Camp,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,Albany,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.083333,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [221]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [241]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['suburbs']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['suburbs'] = manhattan_grouped['suburbs']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,suburbs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany,Café,Stadium,Indian Restaurant,Bubble Tea Shop,Convenience Store,Restaurant,Steakhouse,Playground,Italian Restaurant,Japanese Restaurant
1,Algies Bay,Nature Preserve,Zoo,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Spot,Fish Market,Fish & Chips Shop
2,Anawhata,Hotel,Beach,Zoo,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Spot
3,Ararimu,Convenience Store,Construction & Landscaping,Garden Center,Zoo,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Spot
4,Arch Hill,Café,Bakery,Bar,Zoo,Farm,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market


### Cluster suburbs

In [314]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('suburbs', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 4, 2, 2, 0, 2, 4], dtype=int32)

In [251]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [315]:

manhattan_merged1 = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged1 = manhattan_merged1.join(neighborhoods_venues_sorted.set_index('suburbs'), on='suburbs',how='inner')

manhattan_merged1.tail() # check the last columns!

,suburbs,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
303,Waiwera,-36.543633,174.706072,1,Beach,Hill,Bar,Southern / Soul Food Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Zoo,Falafel Restaurant
304,Warkworth,-36.398491,174.664232,4,Café,Pub,Intersection,Grocery Store,Sandwich Place,Zoo,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant
305,Whangaparaoa,-36.606554,174.820682,2,Summer Camp,Zoo,Cricket Ground,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Spot,Fish Market,Fish & Chips Shop
307,Whangateau,-36.312665,174.765770,2,Music Venue,Athletics & Sports,Campground,Zoo,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Spot
309,Wellsford,-36.294967,174.522984,4,Fast Food Restaurant,Café,Grocery Store,Bakery,Sandwich Place,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Zoo


#### Map clusters

In [316]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged1['latitude'], manhattan_merged1['longitude'], manhattan_merged1['suburbs'], manhattan_merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Check for current restaurant suburbs


In [350]:
df2=pd.read_csv('restaurant.csv')

In [374]:
df2=df2[['Name','Rating','Review']]
df2

,Name,Rating,Review
0,Thai Isaan Restaurant,3.4,22
1,Bar Africa,2.6,37
2,Mayfield Sportshouse,3.9,71
3,Dutch Delight Ltd,4.3,255
4,Zoo Eatery Lounge,3.6,165
5,"Tony's Restaurant & Bar, Henderson",3.8,234


#### Check for cluster contain Birkenhead, Rosedale, Wairau, Henderson

In [300]:
manhattan_merged1.loc[((manhattan_merged1['suburbs'] == 'Henderson') | (manhattan_merged1['suburbs'] == 'Birkenhead'))| (manhattan_merged1['suburbs'] == 'Rosedale')| (manhattan_merged1['suburbs'] == 'Wairau Valley')]

,suburbs,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
79,Birkenhead,-36.798710,174.712476,4,Pub,Café,Cupcake Shop,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Spot,Fish Market,Fish & Chips Shop
108,Rosedale,-36.737099,174.720972,2,Chinese Restaurant,Sporting Goods Shop,Gym,Korean Restaurant,Café,Playground,Other Great Outdoors,Fishing Spot,Fish Market,Flea Market
221,Henderson,-36.880555,174.633371,2,Department Store,Furniture / Home Store,Sandwich Place,Electronics Store,Toy / Game Store,Grocery Store,Train Station,Bakery,Fast Food Restaurant,Paper / Office Supplies Store


#### Get lists of all suburbs in Cluster 4 and 2

In [307]:
manhattan_merged1.loc[((manhattan_merged1['Cluster Labels'] == 4) | (manhattan_merged1['Cluster Labels'] == 4))]

,suburbs,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Arch Hill,-36.866092,174.745972,4,Café,Bakery,Bar,Zoo,Farm,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
11,Avondale,-36.893058,174.692814,4,Café,Fried Chicken Joint,Chinese Restaurant,Market,Zoo,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market
16,Eden Valley,-36.876714,174.751259,4,Café,Bar,Chinese Restaurant,Thrift / Vintage Store,Fast Food Restaurant,Bakery,Restaurant,Gym Pool,Burger Joint,Supermarket
17,Ellerslie,-36.897603,174.815030,4,Café,Pizza Place,Hotel,Chinese Restaurant,Bar,Bakery,Turkish Restaurant,Coffee Shop,Park,Fish & Chips Shop
19,Freemans Bay,-36.853510,174.751183,4,Bar,Café,Italian Restaurant,Japanese Restaurant,Vietnamese Restaurant,Park,Mexican Restaurant,Restaurant,Zoo,Fish & Chips Shop
22,Grafton,-36.862278,174.767751,4,Café,Coffee Shop,Pharmacy,Grocery Store,Pizza Place,Convenience Store,Deli / Bodega,Sushi Restaurant,Automotive Shop,Food
23,Greenlane,-36.893759,174.793962,4,Supermarket,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Café,Restaurant,Fishing Spot,Fish Market,Falafel Restaurant,Fish & Chips Shop
24,Greenwoods Corner,-36.900170,174.773074,4,Café,Japanese Restaurant,Steakhouse,Bakery,Gastropub,Indian Restaurant,Italian Restaurant,Flower Shop,Flea Market,Fishing Spot
25,Grey Lynn,-36.859922,174.736418,4,Café,Bar,Farmers Market,Bakery,Recording Studio,Vietnamese Restaurant,Falafel Restaurant,Coffee Shop,Fish & Chips Shop,Fast Food Restaurant
26,Herne Bay,-36.842507,174.736383,4,Café,Indian Restaurant,Japanese Restaurant,Thai Restaurant,Fast Food Restaurant,Bar,Pharmacy,French Restaurant,Italian Restaurant,American Restaurant


In [308]:
manhattan_merged1.loc[((manhattan_merged1['Cluster Labels'] == 2) | (manhattan_merged1['Cluster Labels'] == 2))]

,suburbs,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Auckland CBD,-36.848911,174.765226,2,Café,Sushi Restaurant,Indian Restaurant,Hotel,Japanese Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Lounge,Pizza Place,Restaurant
12,Balmoral,-36.889205,174.748694,2,Chinese Restaurant,Thai Restaurant,Fast Food Restaurant,Department Store,Dumpling Restaurant,Gym,Park,Coffee Shop,Movie Theater,Japanese Restaurant
14,Blockhouse Bay,-36.919777,174.701470,2,Ice Cream Shop,Video Store,Café,Neighborhood,Mediterranean Restaurant,Fish & Chips Shop,Grocery Store,Donut Shop,Filipino Restaurant,Food & Drink Shop
15,Eden Terrace,-36.865249,174.756593,2,Café,Indian Restaurant,Bakery,Burger Joint,French Restaurant,Brewery,Sushi Restaurant,Music Venue,Cocktail Bar,Thai Restaurant
18,Epsom,-36.885845,174.773462,2,Chinese Restaurant,Café,Asian Restaurant,Japanese Restaurant,Tailor Shop,Fish & Chips Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market
21,Glen Innes,-36.875526,174.859947,2,Supermarket,Fish & Chips Shop,Food,Pizza Place,Turkish Restaurant,Health & Beauty Service,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
28,Kingsland,-36.871195,174.743936,2,Bar,Middle Eastern Restaurant,Thai Restaurant,Stadium,Bakery,Burger Joint,Café,Mexican Restaurant,Sandwich Place,Tapas Restaurant
29,Kohimarama,-36.855758,174.840233,2,Fish & Chips Shop,Stadium,Italian Restaurant,Pizza Place,Park,Zoo,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
30,Lynfield,-36.927373,174.718848,2,Sandwich Place,Burger Joint,Pizza Place,Grocery Store,Bar,Shopping Mall,Fast Food Restaurant,Zoo,Falafel Restaurant,Farm
32,Mission Bay,-36.849862,174.833645,2,Café,Italian Restaurant,Ice Cream Shop,Fish & Chips Shop,Indian Restaurant,Mexican Restaurant,Burger Joint,Movie Theater,Fast Food Restaurant,Scenic Lookout


## Let assume he wants to open in Auckland city first, get list of all restaurants in Auckland CBD

In [322]:
## check for current restaurant
url='https://developers.zomato.com/api/v2.1/search?user-key=&entity_type=Auckland&lat=-36.848911&lon=174.765226'


In [332]:
header = {"User-agent": "curl/7.43.0", "Accept": "application/json", "user_key": "257bfefdf35d8bcae108f9fe1e30eec4"}

In [328]:
results = requests.get(url,headers=header).json()
results

{'results_found': 6698,
 'results_start': 0,
 'results_shown': 20,
 'restaurants': [{'restaurant': {'R': {'has_menu_status': {'delivery': -1,
      'takeaway': -1},
     'res_id': 18393092,
     'is_grocery_store': False},
    'apikey': '257bfefdf35d8bcae108f9fe1e30eec4',
    'id': '18393092',
    'name': 'Amano Restaurant',
    'url': 'https://www.zomato.com/auckland/amano-restaurant-britomart?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
    'location': {'address': '66-68 Tyler Street, Britomart, Auckland CBD, Auckland',
     'locality': 'Britomart',
     'city': 'Auckland',
     'city_id': 70,
     'latitude': '-36.8446555337',
     'longitude': '174.7706881911',
     'zipcode': '',
     'country_id': 148,
     'locality_verbose': 'Britomart, Auckland'},
    'switch_to_order_menu': 0,
    'cuisines': 'Italian',
    'timings': '7am – 9:30pm (Mon, Tue, Wed, Thu, Sun), 7am – 10pm (Fri-Sat)',
    'average_cost_for_two': 90,
    'price_range': 4,
    'currency': '$',
    '

In [331]:
df3 = pd.json_normalize(results["restaurants"])
df3.head()

,restaurant.R.has_menu_status.delivery,restaurant.R.has_menu_status.takeaway,restaurant.R.res_id,restaurant.R.is_grocery_store,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,restaurant.location.city_id,restaurant.location.latitude,restaurant.location.longitude,restaurant.location.zipcode,restaurant.location.country_id,restaurant.location.locality_verbose,restaurant.switch_to_order_menu,restaurant.cuisines,restaurant.timings,restaurant.average_cost_for_two,restaurant.price_range,restaurant.currency,restaurant.highlights,restaurant.offers,restaurant.opentable_support,restaurant.is_zomato_book_res,restaurant.mezzo_provider,restaurant.is_book_form_web_view,restaurant.book_form_web_view_url,restaurant.book_again_url,restaurant.thumb,restaurant.user_rating.aggregate_rating,restaurant.user_rating.rating_text,restaurant.user_rating.rating_color,restaurant.user_rating.rating_obj.title.text,restaurant.user_rating.rating_obj.bg_color.type,restaurant.user_rating.rating_obj.bg_color.tint,restaurant.user_rating.votes,restaurant.all_reviews_count,restaurant.photos_url,restaurant.photo_count,restaurant.menu_url,restaurant.featured_image,restaurant.has_online_delivery,restaurant.is_delivering_now,restaurant.store_type,restaurant.include_bogo_offers,restaurant.deeplink,restaurant.is_table_reservation_supported,restaurant.has_table_booking,restaurant.events_url,restaurant.phone_numbers,restaurant.all_reviews.reviews,restaurant.establishment,restaurant.establishment_types,restaurant.medio_provider,restaurant.book_url,restaurant.zomato_events
0,-1,-1,18393092,False,257bfefdf35d8bcae108f9fe1e30eec4,18393092,Amano Restaurant,https://www.zomato.com/auckland/amano-restaura...,"66-68 Tyler Street, Britomart, Auckland CBD, A...",Britomart,Auckland,70,-36.8446555337,174.7706881911,,148,"Britomart, Auckland",0,Italian,"7am – 9:30pm (Mon, Tue, Wed, Thu, Sun), 7am – ...",90,4,$,"[Lunch, Cash, Credit Card, Debit Card, Dinner,...",[],0,0,OTHER,0,,,https://b.zmtcdn.com/data/pictures/2/18393092/...,4.8,Excellent,3F7E00,4.8,lime,800,839,325,https://www.zomato.com/auckland/amano-restaura...,1023,https://www.zomato.com/auckland/amano-restaura...,https://b.zmtcdn.com/data/pictures/2/18393092/...,0,0,,True,zomato://restaurant/18393092,0,0,https://www.zomato.com/auckland/amano-restaura...,09 3941416,"[{'review': []}, {'review': []}, {'review': []...",[Casual Dining],[],NaN,NaN,NaN
1,-1,-1,7003663,False,257bfefdf35d8bcae108f9fe1e30eec4,7003663,Federal Delicatessen,https://www.zomato.com/auckland/federal-delica...,"86 Federal Street, Auckland CBD, Auckland 1010",Federal Street,Auckland,70,-36.8490919026,174.7627210245,1010,148,"Federal Street, Auckland",0,American,7am – 9:30pm (Mon-Sun),60,3,$,"[Dinner, Takeaway Available, Breakfast, Debit ...",[],0,0,OTHER,0,,,https://b.zmtcdn.com/data/pictures/3/7003663/7...,4.7,Excellent,3F7E00,4.7,lime,800,1282,616,https://www.zomato.com/auckland/federal-delica...,1671,https://www.zomato.com/auckland/federal-delica...,https://b.zmtcdn.com/data/pictures/3/7003663/7...,0,0,,True,zomato://restaurant/7003663,0,0,https://www.zomato.com/auckland/federal-delica...,021 329228,"[{'review': []}, {'review': []}, {'review': []...",[Casual Dining],[],NaN,NaN,NaN
2,-1,-1,7000700,False,257bfefdf35d8bcae108f9fe1e30eec4,7000700,Pocha 포차,https://www.zomato.com/auckland/pocha-chancery...,"Chancery Square, 2A /48 Courthouse Lane, Auckl...","Chancery Square, Auckland CBD",Auckland,70,-36.8485625569,174.7678738832,1010,148,"Chancery Square, Auckland CBD, Auckland",0,"Korean, Fried Chicken",5:30pm – 12midnight (Mon-Sun),60,3,$,"[Dinner, Takeaway Available, Credit Card, Serv...",[],0,0,OTHER,0,,,https://b.zmtcdn.com/data/pictures/0/7000700/a...,4.4,Very Good,5BA829,4.4,lime,700,668,256,https://www.zomato.com/auckland/pocha-chancery...,547,https://www.zomato.com/auckland/pocha-chancery...,https://b.zmtcdn.com/data/pictures/0/7000700/a...,0,0,,True,zomato://res

#### Get relevant data

In [334]:
df3[['restaurant.name','restaurant.average_cost_for_two','restaurant.price_range','restaurant.user_rating.aggregate_rating','restaurant.user_rating.votes','restaurant.all_reviews_count']]

,restaurant.name,restaurant.average_cost_for_two,restaurant.price_range,restaurant.user_rating.aggregate_rating,restaurant.user_rating.votes,restaurant.all_reviews_count
0,Amano Restaurant,90,4,4.8,839,325
1,Federal Delicatessen,60,3,4.7,1282,616
2,Pocha 포차,60,3,4.4,668,256
3,Depot Eatery and Oyster Bar,90,4,4.9,1025,463
4,Eden Noodles Cafe 四川担担面,35,2,4.5,860,385
5,The Garden Shed,90,4,4.4,480,293
6,Baduzzi,120,4,4.7,703,385
7,The Kimchi Project,60,3,4.3,1022,472
8,The Candy Shop,60,3,4.3,416,184
9,Bambina Newmarket,50,3,4.4,461,257


#### Due to the limit we will get data from CSV

In [354]:
df4=pd.read_csv("aucklandzomato.csv")
df4.head()

,url,Result Title,Result Title_link,Rating Value,Review Count,Search Page,Search Page_link,Col Search 1,Col Nowrap 1,Col Nowrap 1_link,Hours,_url_input
0,https://www.zomato.com/auckland/auckland-centr...,Beard Papa's,https://www.zomato.com/auckland/beard-papas-mo...,3.6,(129 Reviews),"Westfield St Lukes, Mt Albert",https://www.zomato.com/auckland/restaurants/in...,"Westfield St. Lukes, Level 1, 80 St. Lukes Roa...",Japanese,https://www.zomato.com/auckland/restaurants/ja...,"9am – 6pm (Mon, Tue, Wed, Sat), 9am – 9pm...",https://www.zomato.com/auckland/auckland-centr...
1,https://www.zomato.com/auckland/auckland-centr...,Allpress Coffee,https://www.zomato.com/auckland/all-press-coff...,3.6,(20 Reviews),Ponsonby,https://www.zomato.com/auckland/ponsonby-resta...,"266 Ponsonby Road, Ponsonby, Auckland",Coffee and Tea,https://www.zomato.com/auckland/restaurants/co...,"8am – 5pm (Mon-Sat),8am – 4pm (Sun)",https://www.zomato.com/auckland/auckland-centr...
2,https://www.zomato.com/auckland/auckland-centr...,Silver Lake Deep Fried Bread,https://www.zomato.com/auckland/silver-lake-de...,3.6,(16 Reviews),Sandringham,https://www.zomato.com/auckland/sandringham-re...,"211-215 Sandringham Road, Sandringham, Auckland",Chinese,https://www.zomato.com/auckland/restaurants/ch...,6am – 5pm (Mon-Sun),https://www.zomato.com/auckland/auckland-centr...
3,https://www.zomato.com/auckland/auckland-centr...,Hum Cafe,https://www.zomato.com/auckland/hum-cafe-grafton,3.5,(7 Reviews),Grafton,https://www.zomato.com/auckland/grafton-restau...,"123 Grafton Road, Grafton, Auckland 1023",Coffee and Tea,https://www.zomato.com/auckland/restaurants/co...,"Closed (Mon-Thu),10am – 7pm (Fri-Sun)",https://www.zomato.com/auckland/auckland-centr...
4,https://www.zomato.com/auckland/auckland-centr...,Eighthirty,https://www.zomato.com/auckland/eighthirty-pon...,3.6,(11 Reviews),"Ponsonby Central, Ponsonby",https://www.zomato.com/auckland/restaurants/in...,"Ponsonby Central, 4 Brown Street, Ponsonby, Au...",Coffee and Tea,https://www.zomato.com/auckland/restaurants/co...,"8am – 4pm (Mon-Fri),8am – 5pm (Sat-Sun)",https://www.zomato.com/auckland/auckland-centr...


In [365]:
df4[['Result Title','Rating Value','Review Count']]

,Result Title,Rating Value,Review Count
0,Beard Papa's,3.6,(129 Reviews)
1,Allpress Coffee,3.6,(20 Reviews)
2,Silver Lake Deep Fried Bread,3.6,(16 Reviews)
3,Hum Cafe,3.5,(7 Reviews)
4,Eighthirty,3.6,(11 Reviews)
5,Dear Deer Coffee Roasting Bar,NaN,NaN
6,Red Rabbit Coffee Co.,3.0,(11 Reviews)
7,Fount Coffee,NaN,NaN
8,Miscela Coffee House,NaN,NaN
9,Bow And Tie Micro Roastery,NaN,NaN


In [368]:
#drop nan
df4 = df4[df4['Review Count'].notna()]

In [369]:
# extract review number
df4['Review']=df4['Review Count'].str.extract('(\d+)').astype(int)

/Users/andy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [377]:
df5=df4[['Result Title','Rating Value','Review']]
df5.head()

,Result Title,Rating Value,Review
0,Beard Papa's,3.6,129
1,Allpress Coffee,3.6,20
2,Silver Lake Deep Fried Bread,3.6,16
3,Hum Cafe,3.5,7
4,Eighthirty,3.6,11


In [384]:
# rename colume before merging
df5=df5.rename(columns={'Result Title':'Name','Rating Value':'Rating'})

In [445]:
# merge
concatenated = df5.append(df2)
# reset index
concatenated=concatenated.reset_index()
#delete old index
del concatenated['index']

In [446]:
#check if all good
concatenated.tail()

,Name,Rating,Review
640,Bar Africa,2.6,37
641,Mayfield Sportshouse,3.9,71
642,Dutch Delight Ltd,4.3,255
643,Zoo Eatery Lounge,3.6,165
644,"Tony's Restaurant & Bar, Henderson",3.8,234


In [447]:
#add colume for reference
concatenated['like']='0'

In [448]:
concatenated.tail()

,Name,Rating,Review,like
640,Bar Africa,2.6,37,0
641,Mayfield Sportshouse,3.9,71,0
642,Dutch Delight Ltd,4.3,255,0
643,Zoo Eatery Lounge,3.6,165,0
644,"Tony's Restaurant & Bar, Henderson",3.8,234,0


In [450]:
# set current restaurant=1
concatenated.iloc[639:645,3]='1'

In [451]:
concatenated.tail()

,Name,Rating,Review,like
640,Bar Africa,2.6,37,1
641,Mayfield Sportshouse,3.9,71,1
642,Dutch Delight Ltd,4.3,255,1
643,Zoo Eatery Lounge,3.6,165,1
644,"Tony's Restaurant & Bar, Henderson",3.8,234,1


### Apply kmeans to find similar restaurant

In [453]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = concatenated.drop('Name',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 1, 1, 0, 4, 4], dtype=int32)

In [454]:
concatenated.insert(0, 'Cluster Labels', kmeans.labels_)

In [455]:
concatenated.tail()

,Cluster Labels,Name,Rating,Review,like
640,1,Bar Africa,2.6,37,1
641,4,Mayfield Sportshouse,3.9,71,1
642,3,Dutch Delight Ltd,4.3,255,1
643,0,Zoo Eatery Lounge,3.6,165,1
644,3,"Tony's Restaurant & Bar, Henderson",3.8,234,1


### As there are more number of cluster 3, we will get a list of all restaurant in cluster 3 to recommend

In [459]:
concatenated[concatenated['Cluster Labels']==3]

,Cluster Labels,Name,Rating,Review,like
27,3,CrunchTower Desserts & Drinks Room,4.2,322,0
28,3,Petal Cupcakes,4.4,231,0
110,3,Fukurou Patisserie,4.5,274,0
113,3,Ben & Jerry's,4.3,250,0
200,3,Best Ugly Bagels,4.6,363,0
359,3,No 1 Chicken 넘버 원 치킨,4.2,307,0
488,3,Tiger Burger,4.6,437,0
491,3,Cereal Killa Cafe,3.7,424,0
493,3,Ramen DO らーめん道,4.4,286,0
494,3,Ramen Takara,4.1,249,0


## Thank you for reading